Agentic RAG from: https://medium.com/the-ai-forum/multi-document-agentic-rag-using-llama-index-and-mistral-b334fa45d3ee

In [1]:
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,SummaryIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool,QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters,FilterCondition
from typing import List,Optional

In [2]:
import  nest_asyncio
nest_asyncio.apply()

In [3]:
documents = SimpleDirectoryReader(input_files = ['./data/Utel_Espana_Fichas_Tecnicas_Grado_Ingenieria_Industrial_e5cfa85380.pdf']).load_data()
print(len(documents))
print(f"Document Metadata: {documents[0].metadata}")

3
Document Metadata: {'page_label': '1', 'file_name': 'Utel_Espana_Fichas_Tecnicas_Grado_Ingenieria_Industrial_e5cfa85380.pdf', 'file_path': 'data\\Utel_Espana_Fichas_Tecnicas_Grado_Ingenieria_Industrial_e5cfa85380.pdf', 'file_type': 'application/pdf', 'file_size': 372978, 'creation_date': '2024-07-04', 'last_modified_date': '2024-06-18'}


In [4]:
splitter = SentenceSplitter(chunk_size=1024,chunk_overlap=100)
nodes = splitter.get_nodes_from_documents(documents)
print(f"Length of nodes : {len(nodes)}")
print(f"get the content for node 0 :{nodes[0].get_content(metadata_mode='all')}")

Length of nodes : 3
get the content for node 0 :page_label: 1
file_name: Utel_Espana_Fichas_Tecnicas_Grado_Ingenieria_Industrial_e5cfa85380.pdf
file_path: data\Utel_Espana_Fichas_Tecnicas_Grado_Ingenieria_Industrial_e5cfa85380.pdf
file_type: application/pdf
file_size: 372978
creation_date: 2024-07-04
last_modified_date: 2024-06-18

Grado en
Ingeniería Industrial
Sobre el grado
Con este grado, tu potencial te llevará a diseñar 
todos los procesos de una empresa, inﬂuir en todas sus áreas y sacar lo mejor de cada colaborador y colaboradora utilizando síntesis de la ingeniería y el diseño para evaluar y mejorar los resultados. Esta ingeniería busca instruirte en el equipamiento, energía, materiales y procesos de producción de alta calidad y servicios útiles con alta consideración al medio ambiente.
Dónde podrás trabajar
Con
trol de calidad
Asegura el cumplimiento de todas las características y herramientas de productos y servicios.
Procesos y manufactura
Mejora los procesos y propón estra

Instantiate the vectorstore

In [5]:
import chromadb
db = chromadb.PersistentClient(path="./chroma_db_mistral")
chroma_collection = db.get_or_create_collection("multidocument-agent")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Instantiate the embedding model

In [6]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings
#
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
#
Settings.embed_model = embed_model
#
Settings.chunk_size = 1024
#

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

c:\Users\gabriel.hernan\Desktop\papeles personales\Deeplearning\AgenticRAG\env\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gabriel.hernan\AppData\Local\Temp\fastembed_cache\models--qdrant--bge-small-en-v1.5-onnx-q. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

Instantiate the LLM

In [7]:
from llama_index.llms.mistralai import MistralAI
from helper import get_mistral_api_key
MISTRAL_API_KEY = get_mistral_api_key()
llm = MistralAI(model="mistral-large-latest")

Instantiate the Vector Query tool and summary tool for specific document

In [8]:
#instantiate Vectorstore
name = "BERT_arxiv"
vector_index = VectorStoreIndex(nodes,storage_context=storage_context)
vector_index.storage_context.vector_store.persist(persist_path="/content/chroma_db")
#
# Define Vectorstore Autoretrieval tool
def vector_query(query:str,page_numbers:Optional[List[str]]=None)->str:
  '''
  perform vector search over index on
  query(str): query string needs to be embedded
  page_numbers(List[str]): list of page numbers to be retrieved,
                          leave blank if we want to perform a vector search over all pages
  '''
  page_numbers = page_numbers or []
  metadata_dict = [{"key":'page_label',"value":p} for p in page_numbers]
  #
  query_engine = vector_index.as_query_engine(similarity_top_k =2,
                                              filters = MetadataFilters.from_dicts(metadata_dict,
                                                                                    condition=FilterCondition.OR)
                                              )
  #
  response = query_engine.query(query)
  return response
#
#llamiondex FunctionTool wraps any python function we feed it
vector_query_tool = FunctionTool.from_defaults(name=f"vector_tool_{name}",
                                              fn=vector_query)
# Prepare Summary Tool
summary_index = SummaryIndex(nodes)
summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize",
                                                      se_async=True,)
summary_query_tool = QueryEngineTool.from_defaults(name=f"summary_tool_{name}",
                                                    query_engine=summary_query_engine,
                                                  description=("Use ONLY IF you want to get a holistic summary of the documents."
                                              "DO NOT USE if you have specified questions over the documents."))

Test the LLM

In [9]:
response = llm.predict_and_call([vector_query_tool],
                                "Summarize the content in page number 2",
                                verbose=True)

=== Calling Function ===
Calling function: vector_tool_BERT_arxiv with args: {"query": "summarize the content", "page_numbers": ["2"]}
=== Function Output ===
The academic validity of the Industrial Engineering degree program at Utel is recognized by the Mexican National Education System. The program offers concentrations in Administration, Information Technologies, and Finance. It consists of 300 credits covering subjects such as industry structure, algebra, sustainable development, physics, statistics, and more. Students choose two professional paths within their concentration area and complete four co-curricular subjects. The program duration is typically four years internationally, with the option for a more accelerated pace. Additionally, students must take three Advanced Workshops focusing on Employability, Social Impact, and Digital Skills to enhance the quality of their education.


Helper function to generate Vectorstore Tool and Summary tool for all the documents

In [ ]:
def get_doc_tools(file_path:str,name:str)->str:
  '''
  get vector query and sumnmary query tools from a document
  '''
  #load documents
  documents = SimpleDirectoryReader(input_files = [file_path]).load_data()
  print(f"length of nodes")
  splitter = SentenceSplitter(chunk_size=1024,chunk_overlap=100)
  nodes = splitter.get_nodes_from_documents(documents)
  print(f"Length of nodes : {len(nodes)}")
  #instantiate Vectorstore
  vector_index = VectorStoreIndex(nodes,storage_context=storage_context)
  vector_index.storage_context.vector_store.persist(persist_path="/content/chroma_db")
  #
  # Define Vectorstore Autoretrieval tool
  def vector_query(query:str,page_numbers:Optional[List[str]]=None)->str:
    '''
    perform vector search over index on
    query(str): query string needs to be embedded
    page_numbers(List[str]): list of page numbers to be retrieved,
                            leave blank if we want to perform a vector search over all pages
    '''
    page_numbers = page_numbers or []
    metadata_dict = [{"key":'page_label',"value":p} for p in page_numbers]
    #
    query_engine = vector_index.as_query_engine(similarity_top_k =2,
                                                filters = MetadataFilters.from_dicts(metadata_dict,
                                                                                     condition=FilterCondition.OR)
                                                )
    #
    response = query_engine.query(query)
    return response
  #
  #llamiondex FunctionTool wraps any python function we feed it
  vector_query_tool = FunctionTool.from_defaults(name=f"vector_tool_{name}",
                                                fn=vector_query)
  # Prepare Summary Tool
  summary_index = SummaryIndex(nodes)
  summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize",
                                                       se_async=True,)
  summary_query_tool = QueryEngineTool.from_defaults(name=f"summary_tool_{name}",
                                                     query_engine=summary_query_engine,
                                                    description=("Use ONLY IF you want to get a holistic summary of the documents."
                                                "DO NOT USE if you have specified questions over the documents."))
  return vector_query_tool,summary_query_tool

Prepare a input list with specified document names

In [ ]:
import os
root_path = "/content/data"
file_name = []
file_path = []
for files in os.listdir(root_path):
  if file.endswith(".pdf"):
    file_name.append(files.split(".")[0])
    file_path.append(os.path.join(root_path,file))
#
print(file_name)
print(file_path)

Generate the vectortool and summary tool for each documents

In [ ]:
papers_to_tools_dict = {}
for name,filename in zip(file_name,file_path):
  vector_query_tool,summary_query_tool = get_doc_tools(filename,name)
  papers_to_tools_dict[name] = [vector_query_tool,summary_query_tool]

Get the tools into a flat list

In [ ]:
initial_tools = [t for f in file_name for t in papers_to_tools_dict[f]]
initial_tools

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex
#
obj_index = ObjectIndex.from_objects(initial_tools,index_cls=VectorStoreIndex)
#

Set up the ObjectIndex as retriever

In [ ]:
obj_retriever = obj_index.as_retriever(similarity_top_k=2)
tools = obj_retriever.retrieve("compare and contrast the papers self rag and corrective rag")
#
print(tools[0].metadata)
print(tools[1].metadata)

Setup the RAG Agent

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner
#
agent_worker = FunctionCallingAgentWorker.from_tools(tool_retriever=obj_retriever,
                                                     llm=llm,
                                                     system_prompt="""You are an agent designed to answer queries over a set of given papers.
                                                     Please always use the tools provided to answer a question.Do not rely on prior knowledge.""",
                                                     verbose=True)
agent = AgentRunner(agent_worker)

Ask Query 1

In [ ]:
#
response = agent.query("Compare and contrast self rag and crag.")
print(str(response))

In [ ]:
response = agent.query("Summarize the paper corrective RAG.")
print(str(response))